In [5]:
#j ai essaye de mettre plusieurs fonction pour le preprocessing , a choisir , modifier , supprimer 
import os
import glob
import numpy as np
from PIL import Image
import pytesseract
from skimage import io
from skimage.util import img_as_ubyte
from skimage.transform import rotate, resize
from skimage.filters import median, unsharp_mask, threshold_otsu
from skimage.morphology import disk
from skimage import exposure
from skimage.feature import canny
from skimage.transform import hough_line, hough_line_peaks
from typing import Tuple
import cv2
from deskew import determine_skew


def reduce_black_pixels(image, threshold: int = 10):# ca marche pas 
    # Ensure the image is in grayscale
    if len(image.shape) == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image
    # Apply histogram equalization to improve contrast
    equalized_image = cv2.equalizeHist(gray_image)
    
    # Apply a threshold to reduce black pixels
    _, thresholded_image = cv2.threshold(equalized_image, threshold, 255, cv2.THRESH_TOZERO)
    
    return thresholded_image

def reduce_noise(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    denoised_gray = cv2.fastNlMeansDenoising(gray_image, None, 3, 7, 21)
    denoised_image = cv2.cvtColor(denoised_gray, cv2.COLOR_GRAY2BGR)
    return denoised_image

def enhance_contrast(image):
    # Adjust the brightness and contrast 
# Adjusts the brightness by adding 10 to each pixel value 
    brightness = -5 
# Adjusts the contrast by scaling the pixel values by 2.3 
    contrast = 1.5  
    image2= cv2.addWeighted(image, contrast, np.zeros(image.shape, image.dtype), 0, brightness)
    return image2

def filter_image(image):
    kernel = np.array([[0, -1, 0],
                  [-1, 5,-1],
                  [0, -1, 0]])
    sharpen = cv2.filter2D(src=image, ddepth=-1, kernel=kernel)
    return sharpen


def preprocess_image(image):
    #image = reduce_noise(image)
    #image = enhance_contrast(image)
    #image = filter_image(image)
    #image =reduce_black_pixels(image)
    image =reduce_black_pixels(image)
    return image

def resize_image(image):
    return cv2.resize(image, (1000, 1000)) 

def get_orientation(file):
    image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    edges = cv2.Canny(image, 50, 150, apertureSize=3)
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 200)
    angles = []
    if lines is not None:
        for rho, theta in lines[:, 0]:
            angle = np.degrees(theta) - 90
            angles.append(angle)
        angle_moyen = np.mean(angles)
        return angle_moyen
    else:
        return None
def correct_rotation(image: np.ndarray, confidence_threshold: int = 12):
    pil_image = Image.fromarray(image)
    osd = pytesseract.image_to_osd(pil_image, output_type=pytesseract.Output.DICT)
    rotate_angle = osd['rotate']
    orientation_conf = osd['orientation_conf']
    
    if orientation_conf < confidence_threshold:
        rotate_angle = 0
    
    corrected_pil_image = pil_image.rotate(-rotate_angle, expand=True)
    corrected_image = np.array(corrected_pil_image)
    
    return corrected_image, osd

def deskew(image, resize: bool = True):
    angle = determine_skew(image)
    rotated = rotate(image, angle, resize=resize)
    rotated = np.clip(rotated, 0, 1)
    rotated = img_as_ubyte(rotated)
    return rotated

def process_images(input_directory: str, output_directory: str, batch_size: int, max_images: int):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    image_files = glob.glob(f"{input_directory}/*.tif")[:max_images]
    total_images = len(image_files)
    failed_images_count = 0

    for i in range(0, total_images, batch_size):
        batch_files = image_files[i:i + batch_size]
        for image_file in batch_files:
            try:
                # Load the image
                image = io.imread(image_file)
                 # Resize the image
                image = resize_image(image)
                # Correct the rotation
                image, osd = correct_rotation(image)
                # Deskew the image if applicable
                image = deskew(image)
                # Preprocess the image
                image = preprocess_image(image)
            
                # Save the corrected and deskewed image
                output_file = os.path.join(output_directory, os.path.basename(image_file))
                Image.fromarray(image).save(output_file)

            except Exception:
                # Move the failed image to the output directory
                failed_output_file = os.path.join(output_directory, os.path.basename(image_file))
                io.imsave(failed_output_file,image)
                failed_images_count += 1

    # Calculate and print the percentage of failed images
    failed_percentage = (failed_images_count / total_images) * 100
    print(f"Percentage of failed images: {failed_percentage:.2f}%")

# Configuration
input_directory = 'C:/Users/HP/Desktop/projet/rvl_images'
output_directory = 'C:/Users/HP/Desktop/projet/correct_rvl_images'
batch_size = 100
max_images = 200
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
# Process images in batches
process_images(input_directory, output_directory, batch_size, max_images)


KeyboardInterrupt: 